In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import sys
sys.path.append("src") # go to parent dir
from detect_for_paper import *
from reproject import *
import rosbag
from sklearn.metrics import mean_squared_error
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import cv2

rospy.init_node("sd_jupyter")

[[1.37544824e+03 0.00000000e+00 9.50562012e+02]
 [0.00000000e+00 1.37584045e+03 5.66901917e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


In [3]:
bag = rosbag.Bag("/home/xuhao/bags/dl_data/swarm_dl_one_ground_2019-08-18-05-52-01.bag")
#bag = rosbag.Bag("/home/xuhao/bags/swarm_one_drone_reproject_2019-08-18-15-37-47.bag")
#bag = rosbag.Bag("/home/xuhao/bags/dl_data/swarm_dl_2drone_fly_2019-08-17-12-42-29.bag")
#bag = rosbag.Bag("/media/xuhao/Swarm/swarm_bags/swarm_dl_2drone_fly_2019-08-17-12-42-29.orig.bag")
#bag = rosbag.Bag("/home/xuhao/bags/dl_data/swarm_dl_four_drones_2019-08-14-07-49-37.bag")
poses, camera, depths = read_bag(bag)
dp2 = parse_ros_pose_ts(poses[2])
dp4 = parse_ros_pose_ts(poses[4])

In [4]:
### index = 100
sd = SwarmDetector(debug_show="",conf_thres=0.9,weights_path="/home/xuhao/swarm_ws/src/swarm_yolo/weights/yolov3-tiny_drone.pth")
step = 1
err = []
est_3d_poses = []

gt_3d_poses = []

start_index = 0

print(len(depths))
for index in range(start_index, len(depths), step):
#     print(index - start_index)
    t_cam_on_drone = np.array([0.044, -0.035, 0.0])
    R_cam_on_drone = np.array([[0, 0, 1],
             [-1, 0, 0],
             [0, -1, 0]])
    
    img_msg = camera[index]
    dposyolo_bodyframe = sd.img_callback(img_msg, depths[index])
#     _img  = sd.img_callback(img_msg, depths[index])
#     _img = cv2.resize(_img, (1280, 960))
#     cv2.putText(_img, "FRAME {}".format(index - start_index), (100, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (200, 200, 200), 1, cv2.LINE_AA)
#     cv2.imshow("YOLO", _img)
#     k = cv2.waitKey(10)
    #if k == ord('q'):
    #    break
        
    if dposyolo_bodyframe is None:
        continue
    else:
        pass
        #print(dposyolo_camframe)
    pos4 = dp4["pos_func"](img_msg.header.stamp.to_sec()) + np.array([0., 0.0, 0.0])
    quat4 = dp4["quat_func"](img_msg.header.stamp.to_sec())
    quat2 = dp2["quat_func"](img_msg.header.stamp.to_sec())
    pos2 = dp2["pos_func"](img_msg.header.stamp.to_sec())
    dpos_gt = pos4 - pos2 

    r2 = quaternion_matrix(quat2)[0:3,0:3]
    dposyolo_global = np.dot(r2, dposyolo_bodyframe)
    dyolo_gt_err = dposyolo_global - dpos_gt
    if math.fabs(dyolo_gt_err[0]) > 0.5 or math.fabs(dyolo_gt_err[1]) > 0.5 or math.fabs(dyolo_gt_err[2]) > 0.5:
        continue
    if 1.0 < np.linalg.norm(dpos_gt) < 2.8:
        err.append(dposyolo_global - dpos_gt)
        est_3d_poses.append(dposyolo_global + pos2)
    
    

cv2.destroyAllWindows()
err = numpy.array(err)

Model loaded; waiting for data
2179
[INFO] [1568098374.026318]: Detection use time 11.011124ms
[INFO] [1568098374.028830]: Total use time 16.845942ms
[INFO] [1568098374.032361]: Total use time 2.305031ms
Detect 1.05572330477 [1.0647233  0.00440479 0.01406473]
Detect 1.05097480859 [ 1.05997481 -0.16641274  0.05732777]
Detect 1.08755145284 [ 1.09655145  0.2499671  -0.300298  ]
Detect 1.21840472018 [ 1.22740472  0.59605446 -0.28817423]
Detect 1.2827196292 [ 1.29171963  0.69209762 -0.30474669]
Detect 1.336256602 [ 1.3452566   0.64014009 -0.18354295]
Detect 1.33447031337 [ 1.34347031  0.55836011 -0.13547264]
Detect 1.34444710663 [ 1.35344711  0.53758985 -0.13010057]
Detect 1.36102272149 [ 1.37002272  0.48562641 -0.07201732]
Detect 1.36744412361 [1.37644412e+00 3.94517383e-01 8.00853261e-04]
Detect 1.37295550787 [1.38195551 0.36455199 0.00995369]
Detect 1.3740455836 [1.38304558 0.32230264 0.0231195 ]
Detect 1.38288902259 [1.39188902 0.30548964 0.02899237]
Detect 1.37447003849 [1.38347004 0.2

In [7]:
err = np.array(err)
est_3d_poses = np.array(est_3d_poses )
plt.figure(100, figsize=(6, 6))
plt.clf()

fig, axs = plt.subplots(1, 3, constrained_layout=True)

plt.rcParams.update({'font.size': 24})
axs[0].hist(est_3d_poses[:,0] - np.mean(est_3d_poses[:,0]), density=True, bins=30, range=(-0.3, 0.3))
axs[0].set_xlabel("x[m]")
axs[0].set_ylabel("desnity")
axs[0].grid()
fig.suptitle('Relative Position Error', fontsize=32)

axs[1].hist(est_3d_poses[:,1] - np.mean(est_3d_poses[:,1]), density=True, bins=30, range=(-0.2, 0.2))
axs[1].set_xlabel("y[m]")
axs[1].grid()

axs[2].hist(est_3d_poses[:,2] - np.mean(est_3d_poses[:,2]), density=True, bins=30, range=(-0.15, 0.15))
axs[2].set_xlabel("z[m]")
axs[2].grid()
plt.savefig("detection_accuracy.pdf")

plt.show()

plt.figure(102, figsize=(6, 6))
plt.subplot(131)
plt.hist(est_3d_poses[:,0] - np.mean(est_3d_poses[:,0]), density=True, bins=30, range=(-0.3, 0.3))
plt.xlabel("ERR X")
plt.grid()

plt.subplot(132)
plt.hist(est_3d_poses[:,1] - np.mean(est_3d_poses[:,1]), density=True, bins=30, range=(-0.2, 0.2))
plt.xlabel("ERR Y")
plt.grid()

plt.subplot(133)
plt.hist(est_3d_poses[:,2] - np.mean(est_3d_poses[:,2]), density=True, bins=30, range=(-0.2, 0.2))
plt.xlabel("ERR Z")
plt.grid()
plt.show()
#avg_err = np.mean(err, axis=0)
avg_err = np.zeros(3)
print("XYZ MEAN {:3.1f} {:3.1f} {:3.1f} cm RMSE {:3.1f} {:3.1f} {:3.1f}cm".format(
    np.mean(err[:,0])*100, 
    np.mean(err[:,1])*100, 
    np.mean(err[:,2])*100,
    np.sqrt(np.mean((err[:,0] - avg_err[0])**2))*100,
    np.sqrt(np.mean((err[:,1] - avg_err[1])**2))*100,
    np.sqrt(np.mean((err[:,2] - avg_err[0])**2))*100))

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(est_3d_poses[:,0], est_3d_poses[:,1], est_3d_poses[:,2], marker="^")
#[ 1.16626224, -0.21409655,  0.52796817]
ax.set_xlim3d(0.5, 1.5)
ax.set_ylim3d(-0.5, 0.)
ax.set_zlim3d(0, 1)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")


XYZ MEAN 6.7 -0.0 -2.1 cm RMSE 12.5 9.6 3.4cm


Text(0.5,0,'Z')